In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#install mlxtend package by opening cmd and
#pip install mlxtend

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
df = pd.read_csv('/kaggle/input/market-basket-analysis/france_data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
#number of unique transactions
len(df.groupby(['InvoiceNo'])['Quantity'].count())

In [ ]:
#highest billed customer#
max_buy = pd.DataFrame(df.groupby(['CustomerID'])['UnitPrice'].sum().reset_index())
max_buy = max_buy.sort_values(by='UnitPrice', ascending=False)
max_buy.head()

In [ ]:
#taking a look at the purchases made by customer
highest_cust = df[df['CustomerID']==12536]
highest_cust = highest_cust.sort_values(by='UnitPrice', ascending=False)
highest_cust.head()
#it is seen that there are two products whose Description is Manual and there price is high
#lesson: look at the data carefully before believing it

In [ ]:
#second highest billed customer
sec_cust = df[df['CustomerID']==12681]
sec_cust = sec_cust.sort_values(by='UnitPrice', ascending = False)
sec_cust.head()

In [ ]:
#creating a user*product matrix
basket = pd.pivot_table(df,index='InvoiceNo', columns='Description',values='Quantity', fill_value=0)
basket.shape

In [ ]:
#function to replace natural number with 1
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1) #dropping off the records which have been posted


In [ ]:
#applying apriori algorithm to generate rules which have support more than 0.05
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
frequent_itemsets.head()

In [ ]:
#calculating confidence and lift for the genrated rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

In [ ]:
#selecting only the rules which have lift >6 and confidence >0.8
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

By looking at the rules, it seems that the **green and red alarm clocks are purchased together and the red paper cups, napkins and plates are purchased together** in a manner that is higher than the overall probability would suggest.
At this point, you may want to look at how much opportunity there is to use the popularity of one product to drive sales of another. For instance, we can see that we sell 340 Green Alarm clocks but only 316 Red Alarm Clocks so maybe we can drive more Red Alarm Clock sales through recommendations?